# Tensorflow saver Test 

### restore with different optimizer (feat. adam) on simple MF model

- 2017.1.24

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
Data = np.random.randint(0,5,(200,100))

In [3]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [200,100], name="X")

P = tf.get_variable(
    "P", [200, 10], initializer=tf.random_normal_initializer(0, 0.1))
Q = tf.get_variable(
    "Q", [100, 10], initializer=tf.random_normal_initializer(0, 0.1))

X_pred = tf.matmul(P,tf.transpose(Q))

In [4]:
loss = tf.reduce_mean(tf.squared_difference(X, X_pred))

---

In [5]:
optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

In [6]:
init = tf.global_variables_initializer()

---

## run n save

In [7]:
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    for it in range(1,101):
        _,l,Pp,Qp = sess.run([optimizer,loss,P,Q], feed_dict={X: Data})
        if it%10==0: 
            print(l)
    print(Pp[0][:10])
    print(Qp[0][:10])
            
    try:
        save_path = saver.save(sess, './mf.model')
        print("Model saved.")
    except Exception as err:
        print(err)

6.06848
6.06827
6.06805
6.06784
6.06763
6.06741
6.06719
6.06697
6.06675
6.06652
[ 0.13916092 -0.08124139 -0.12695986  0.08627793 -0.06326085 -0.07543805
  0.14904258  0.11553844 -0.18912892  0.05422319]
[ 0.02826945  0.01856981 -0.24810702 -0.21994434  0.02325869  0.04355475
 -0.03592653 -0.09070697  0.01681091  0.12088958]
Model saved.


## restore n check

In [8]:
saver = tf.train.Saver()
with tf.Session() as sess:
    try:    
        saver.restore(sess, './mf.model')
        print("Model restored.")
        Pp2,Qp2 = sess.run([P,Q])
        print(Pp2[0][:10])
        print(Qp2[0][:10])
    except Exception as err:
        print(err)

INFO:tensorflow:Restoring parameters from ./mf.model
Model restored.
[ 0.13916092 -0.08124139 -0.12695986  0.08627793 -0.06326085 -0.07543805
  0.14904258  0.11553844 -0.18912892  0.05422319]
[ 0.02826945  0.01856981 -0.24810702 -0.21994434  0.02325869  0.04355475
 -0.03592653 -0.09070697  0.01681091  0.12088958]


## restore n check n run n save again

In [9]:
saver = tf.train.Saver()
with tf.Session() as sess:
    try:
#         optimizer = tf.train.AdamOptimizer(0.1).minimize(loss)
#         init = tf.global_variables_initializer()
#         sess.run(init)
        
        saver.restore(sess, './mf.model')
        print("Model restored.")
        Pp2,Qp2 = sess.run([P,Q])
        print(Pp2[0][:10])
        print(Qp2[0][:10])
        
        var_list = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
        opt = tf.train.AdamOptimizer(0.1)
        optimizer = opt.minimize(loss, var_list=var_list)
        reset_opt_op = tf.variables_initializer(
            [opt.get_slot(var, name) for name in opt.get_slot_names() for var in var_list])
        sess.run(reset_opt_op)
        # if the restore optimizer is adam, add these two.
        sess.run(sess.graph.get_tensor_by_name('beta1_power/Assign:0'))
        sess.run(sess.graph.get_tensor_by_name('beta2_power/Assign:0'))
        
        # continue runing
        for it in range(1,101):
            _,l,pred3,Pp3,Qp3 = sess.run([optimizer,loss,X_pred,P,Q], feed_dict={X: Data})
            if it%10==0: 
                print(l)
        print(pred3[0][:10])
        print(Pp3[0][:10])
        print(Qp3[0][:10])
        
        # save again
        try:
            save_path = saver.save(sess, './mf1.model')
            print("Model saved.")
        except Exception as err:
            print(err)
        
    except Exception as err:
        print(err)

INFO:tensorflow:Restoring parameters from ./mf.model
Model restored.
[ 0.13916092 -0.08124139 -0.12695986  0.08627793 -0.06326085 -0.07543805
  0.14904258  0.11553844 -0.18912892  0.05422319]
[ 0.02826945  0.01856981 -0.24810702 -0.21994434  0.02325869  0.04355475
 -0.03592653 -0.09070697  0.01681091  0.12088958]
2.84233
1.97411
1.81552
1.75322
1.66052
1.6228
1.59489
1.57984
1.56665
1.55773
[ 1.27529025  1.64855552  2.97173619  1.38606608  2.96814728  2.63457608
  2.20879436  1.73130977  1.52818847  1.76617002]
[ 0.09651278 -0.94102162 -0.82287133  0.14125878 -0.85094011  0.33515686
 -0.40291825  0.92688352 -0.24264942  0.7725302 ]
[ 0.44930258  0.15636776 -0.21256459 -0.31694606 -0.44654006 -1.27156425
 -0.54710144  0.59498882  0.48333111  0.82650137]
Model saved.
